##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 로컬 오케스트레이터와 함께 템플릿을 사용하여 TFX 파이프라인 생성하기

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table></div>

## 시작하기

이 문서에서는 TFX Python 패키지와 함께 제공되는 *템플릿*을 사용하여 TensorFlow Extended(TFX) 파이프라인을 생성하는 지침을 제공합니다. 지침의 대부분은 Linux 쉘 명령어이며 `!`를 사용하여 해당 명령을 호출하는 주피터 노트북 코드 셀이 제공됩니다.

시카고 시에서 출시된 [Taxi Trips 데이터세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)를 사용하여 파이프라인을 구축할 것입니다. 이 파이프라인을 활용하여 데이터세트를 사용하는 자체 파이프라인을 구축할 것을 권장합니다.

로컬 환경에서 실행되는 파이프라인을 구축할 것입니다. Google Cloud에서 Kubeflow 오케스트레이터를 사용하는 데 관심이 있는 경우 [Cloud AI Platform 파이프라인의 TFX 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)을 참조하세요.

## 전제 조건

- Linux / MacOS
- Python &gt;= 3.5.3

[Google Colab에서 이 노트를 실행](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb)하면 모든 전제 조건을 쉽게 충족할 수 있습니다.


## 1단계. 환경 설정

**이 문서에서는 명령문을 두 번 제시합니다. 한 번은 복사하여 붙여넣기 가능한 셸 명령으로, 한 번은 주피터 노트북 셸 명령으로 제공합니다. Colab을 사용하는 경우 셸 스크립트 블록을 건너뛰고 노트북 셀을 실행하면 됩니다.**

파이프라인을 빌드하려면 개발 환경을 준비해야 합니다.

`tfx` Python 패키지를 설치합니다. 로컬 환경에서 `virtualenv`를 사용하는 것이 좋습니다. 다음 셸 스크립트 스니펫을 사용하여 환경을 설정할 수 있습니다.

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

Colab을 사용하는 경우:


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

참고: 패키지 설치하는 동안 다음과 같은 오류가 일부 발생할 수 있습니다.

> ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

지금은 이러한 오류를 무시하고 계속 진행합니다.

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

TFX 버전을 확인합니다.

```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

이제 파이프라인을 생성할 수 있습니다.

## 2단계. 미리 정의된 템플릿을 프로젝트 디렉터리에 복사

이 단계에서는 미리 정의된 템플릿에서 추가 파일을 복사하여 작업 파이프라인 프로젝트 디렉터리 및 파일을 생성하겠습니다.

아래 `PIPELINE_NAME`을 변경하여 파이프라인에 다른 이름을 지정할 수 있으며, 이는 파일이 저장되는 프로젝트 디렉터리 이름으로도 사용됩니다.

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX에는 TFX Python 패키지가 있는 `taxi` 템플릿이 포함되어 있습니다. 분류 및 회귀 등의 포인트별 예측 문제를 해결하려는 경우 이 템플릿으로 시작할 수 있습니다.

`tfx template copy` CLI 명령은 미리 정의된 템플릿 파일을 프로젝트 디렉토리에 복사합니다.

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

이 노트북의 작업 디렉토리 컨텍스트를 프로젝트 디렉토리로 변경합니다.

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## 3단계. 복사한 소스 파일 탐색

TFX 템플릿은 파이프라인의 출력을 분석하기 위해 Python 소스 코드, 샘플 데이터 및 주피터 노트북을 포함하여 파이프라인을 구축하기 위한 기본 스캐폴드 파일을 제공합니다. `taxi` 템플릿은 <a>Airflow 튜토리얼</a>과 동일한 <em>Chicago Taxi</em> 데이터세트 및 ML 모델을 사용합니다.

Google Colab에서 왼쪽의 폴더 아이콘을 클릭하여 파일을 탐색할 수 있습니다.  이 경우 파일은 이름이 `my_pipeline`인 프로젝트 디렉터리 아래에 복사되어야 합니다. 디렉터리 이름을 클릭하여 디렉터리 내용을 확인한 후 파일 이름을 두 번 클릭하여 열 수 있습니다.

다음은 각 Python 파일에 대한 간략한 소개입니다.

- `pipeline` -이 디렉토리는 파이프라인을 정의합니다.
    - `configs.py` — 파이프라인 러너의 공통 상수를 정의합니다.
    - `pipeline.py` — TFX 컴포넌트 및 파이프라인을 정의합니다.
- `models` - 이 디렉토리는 ML 모델을 정의합니다.
    - `features.py`, `features_test.py` — 모델의 특성을 정의합니다.
    - `preprocessing.py`, `preprocessing_test.py` — `tf::Transform`을 사용하여 전처리 작업을 정의합니다.
    - `estimator` - 이 디렉토리는 Estimator 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — TF estimator를 사용하여 DNN 모델을 정의합니다.
    - `keras` -- 이 디렉토리는 Keras 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — Keras를 사용하여 DNN 모델을 정의합니다.
- `local_runner.py`, `kubeflow_runner.py` — 각 오케스트레이션 엔진의 러너를 정의합니다.


이름에 `_test.py`가 포함된 일부 파일이 있음을 알 수 있습니다. 이는 파이프라인의 단위 테스트이며 자체 파이프라인을 구현할 때 더 많은 단위 테스트를 추가하는 것이 좋습니다. `-m` 플래그와 함께 테스트 파일의 모듈 이름을 제공하여 단위 테스트를 실행할 수 있습니다. 일반적으로 `.py` 확장자를 삭제하고 `/`를 `.`로 대체하여 모듈 이름을 얻을 수 있습니다. 예를 들면 다음과 같습니다.

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 4단계. 첫 번째 TFX 파이프라인 실행

`pipeline create` 명령을 사용하여 파이프라인을 생성할 수 있습니다.

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

그런 다음 `run create` 명령을 사용하여 생성된 파이프라인을 실행할 수 있습니다.

```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

성공하면 `Component CsvExampleGen is finished.`이 표시됩니다. 템플릿을 복사하면 하나의 구성요소인 CsvExampleGen만 파이프라인에 포함됩니다.

## 5단계. 데이터 검증을 위한 구성 요소 추가

`StatisticsGen`, `SchemaGen`, `ExampleValidator`을 포함하여 데이터 검증을 위한 구성 요소를 추가합니다. 데이터 검증에 대해 자세히 알고 싶으면 [Tensorflow 데이터 검증 시작하기](https://www.tensorflow.org/tfx/data_validation/get_started)를 참조하세요.

`pipeline/pipeline.py`에서 복사한 파이프라인의 정의를 수정합니다. 로컬 환경에서 작업하는 경우 선호하는 편집기를 사용하여 파일을 편집합니다. Google Colab에서 작업하는 경우,

> **왼쪽에 있는 폴더 아이콘을 클릭하여 `Files` 보기를 엽니다**.

> **`my_pipeline`을 클릭하여 디렉터리를 열고 `pipeline` 디렉터리를 클릭하여 열고 `pipeline.py`를 두번 클릭하여 파일을 엽니다**.

> 파이프라인에 `StatisticsGen`, `SchemaGen` 및 `ExampleValidator`를 추가하는 3개의 행을 찾아 주석 처리를 제거합니다(팁: `TODO(step 5):`가 포함된 주석을 찾으세요).

> 변경 사항은 몇 초 후에 자동으로 저장됩니다. 탭 제목에서 `pipeline.py` 앞의 `*` 표시가 사라졌는지 확인합니다. **Colab에는 파일 편집기용 저장 버튼이나 바로가기가 없습니다. 파일 편집기의 Python 파일은 `playground` 모드에서도 런타임 환경에 저장할 수 있습니다.**

이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. `tfx pipeline update` 명령을 사용하여 파이프라인을 업데이트한 다음 `tfx run create` 명령을 사용하여 업데이트된 파이프라인의 새 실행을 생성합니다.

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

추가된 구성 요소의 출력 로그를 볼 수 있게 됩니다. 우리의 파이프라인은 `tfx_pipeline_output/my_pipeline` 디렉터리에 출력 아티팩트를 생성합니다.

## 6단계. 훈련용 구성 요소 추가

이 단계에서는 `Transform`, `Trainer`, `Resolver`, `Evaluator`, `Pusher`를 포함하여 훈련 및 모델 검증을 위한 구성 요소를 추가합니다.

> **`pipeline/pipeline.py`를 엽니다**. 파이프라인에 `Transform`, `Trainer`, `Resolver`, `Evaluator`, `Pusher`를 추가하는 5개의 줄을 찾아 주석 처리를 제거합니다(팁: `TODO(step 6):`를 찾으세요).

이전처럼 이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. 지침은 5단계와 동일합니다. `tfx pipeline update`를 사용하여 파이프라인을 업데이트하고 `tfx run create`를 사용하여 실행을 생성합니다.

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

이 실행이 성공적으로 완료되면 로컬 오케스트레이터를 사용하여 TFX 파이프라인을 처음으로 생성하고 실행하게 됩니다.

**참고:** 파이프라인 실행을 생성할 때마다 입력 및 매개변수가 변경되지 않은 경우에도 모든 구성 요소가 계속해서 실행될 수 있습니다. 해당 실행은 파이프라인 캐싱으로 건너뛰어 시간과 리소스를 절약할 수 있습니다. `pipeline.py`의 `Pipeline` 객체에 `enable_cache=True`를 지정하여 캐싱을 활성화할 수 있습니다.


## 7단계. (*선택 사항*) BigQueryExampleGen 사용하기

<a>BigQuery</a>는 확장성이 뛰어나며 비용 효율적인 서버리스 클라우드 데이터 웨어하우스입니다. BigQuery는 TFX에서 훈련 예제 소스로 사용할 수 있습니다. 이 단계에서는 <code>BigQueryExampleGen</code>를 파이프라인에 추가하겠습니다.

BigQuery를 사용하려면 [Google Cloud Platform](https://cloud.google.com/gcp/getting-started) 계정이 필요합니다. GCP 프로젝트를 준비해 주세요.

Colab 인증 라이브러리 또는 `gcloud` 유틸리티를 사용하여 프로젝트에 로그인합니다.

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


TFX를 사용하여 BigQuery 리소스에 액세스하려면 GCP 프로젝트 이름을 지정해야 합니다. `GOOGLE_CLOUD_PROJECT` 환경 변수를 프로젝트 이름으로 설정합니다.

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **`pipeline/pipeline.py`를 엽니다.** `CsvExampleGen`을 주석 처리하고 `BigQueryExampleGen`의 인스턴스를 생성하는 줄의 주석 처리를 제거합니다. 또한 `create_pipeline` 함수의 `query` 인수에 대한 주석 처리를 제거해야 합니다.

BigQuery에 사용할 GCP 프로젝트를 지정해야 하며, 이를 위해 파이프라인 생성 시 `beam_pipeline_args`에서 `--project`를 설정합니다.

> **`pipeline/configs.py`를 엽니다.** `BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS`와 `BIG_QUERY_QUERY` 정의의 주석 처리를 제거합니다. 이 파일의 프로젝트 ID와 지역 값을 GCP 프로젝트의 올바른 값으로 교체해야 합니다.

> **`local_runner.py`**를 엽니다. create_pipeline() 메서드에서 `query`와 `beam_pipeline_args`라는 두 인자의 주석 처리를 제거합니다.

이제 파이프라인에서 BigQuery를 예제 소스로 사용할 수 있습니다. 5단계 및 6댄계와 같이 파이프라인을 업데이트하고 새 실행을 생성합니다.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## 다음 단계: 파이프라인에 데이터 수집

Chicago Taxi 데이터세트를 사용하여 모델의 파이프라인을 만들었습니다. 이제 데이터를 파이프라인에 입력하겠습니다.

데이터는 GCS 또는 BigQuery를 포함하여 파이프라인이 액세스할 수 있는 모든 곳에 저장할 수 있습니다. 파이프라인 정의를 수정해야 데이터에 액세스할 수 있습니다.

1. 데이터가 파일에 저장된 경우 `kubeflow_runner.py` 또는 `local_runner.py`에서 `DATA_PATH`를 수정하고 해당 파일 위치로 설정합니다. 데이터가 BigQuery에 저장되어 있는 경우, `pipeline/configs.py`의 `BIG_QUERY_QUERY`를 수정하여 데이터를 올바로 쿼리할 수 있습니다.
2. `models/features.py`에서 특성을 추가합니다.
3. `models/preprocessing.py`를 수정하여 [훈련에 적합하게 입력 데이터로 변환](https://www.tensorflow.org/tfx/guide/transform)합니다.
4. `models/keras/model.py` 및 `models/keras/constants.py`를 수정하여 [ML 모델을 설명](https://www.tensorflow.org/tfx/guide/trainer)합니다.
    - Estimator 기반 모델도 사용할 수 있습니다. `pipeline/configs.py`에서 `RUN_FN` 상수를 `models.estimator.model.run_fn`으로 변경합니다.

자세한 소개는 [Trainer 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/trainer)를 참조하세요.